In [2]:
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id="BlinkDL/rwkv-4-pile-14b", filename="RWKV-4-Pile-14B-20230213-8019.pth", cache_dir="/home/kyle/HF-MODEL/rwkv-4-pile-14b")

/home/kyle/miniconda3/envs/kyle-elk-rwkv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/home/kyle/HF-MODEL/rwkv-4-pile-14b/models--BlinkDL--rwkv-4-pile-14b/snapshots/939b6851f96122b7b49bd00d446b3b49481214dd/RWKV-4-Pile-14B-20230213-8019.pth'

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
os.environ["RWKV_JIT_ON"] = '1'
os.environ["RWKV_CUDA_ON"] = '0' # if '1' then use CUDA kernel for seq mode (much faster)
from rwkv.model import RWKV                         # pip install rwkv
model = RWKV(model='/home/kyle/HF-MODEL/rwkv-4-pile-1b5/models--BlinkDL--rwkv-4-pile-1b5/snapshots/6ea995eaa87a17af560c9b41ce1a3d92355c5a49/RWKV-4-Pile-1B5-20220903-8040.pth', strategy='cuda fp16')

out, state = model.forward([187, 510, 1563, 310, 247], None)   # use 20B_tokenizer.json
print(out.detach().cpu().numpy())                   # get logits
out, state = model.forward([187, 510], None)
out, state = model.forward([1563], state)           # RNN has state (use deepcopy if you want to clone it)
out, state = model.forward([310, 247], state)
print(out.detach().cpu().numpy())                   # same result as above

RWKV_JIT_ON 1 RWKV_CUDA_ON 0 RESCALE_LAYER 6

Loading /home/kyle/HF-MODEL/rwkv-4-pile-1b5/models--BlinkDL--rwkv-4-pile-1b5/snapshots/6ea995eaa87a17af560c9b41ce1a3d92355c5a49/RWKV-4-Pile-1B5-20220903-8040.pth ...
Strategy: (total 24+1=25 layers)
* cuda [float16, float16], store 25 layers
0-cuda-float16-float16 1-cuda-float16-float16 2-cuda-float16-float16 3-cuda-float16-float16 4-cuda-float16-float16 5-cuda-float16-float16 6-cuda-float16-float16 7-cuda-float16-float16 8-cuda-float16-float16 9-cuda-float16-float16 10-cuda-float16-float16 11-cuda-float16-float16 12-cuda-float16-float16 13-cuda-float16-float16 14-cuda-float16-float16 15-cuda-float16-float16 16-cuda-float16-float16 17-cuda-float16-float16 18-cuda-float16-float16 19-cuda-float16-float16 20-cuda-float16-float16 21-cuda-float16-float16 22-cuda-float16-float16 23-cuda-float16-float16 24-cuda-float16-float16 
emb.weight                        f16      cpu  50277  2048 
blocks.0.ln1.weight               f16   cuda:0   2048       

In [20]:
from transformers import AutoTokenizer, GPT2TokenizerFast
tokenizer = GPT2TokenizerFast(tokenizer_file='20B_tokenizer.json')
text = "The following movie review expresses what sentiment? George P. Cosmatos\' \"Rambo: First Blood Part II\" is pure wish-fulfillment. The United States clearly didn\'t win the war in Vietnam. They caused damage to this country beyond the imaginable and this movie continues the fairy story of the oh-so innocent soldiers. The only bad guys were the leaders of the nation, who made this war happen. The character of Rambo is perfect to notice this. He is extremely patriotic, bemoans that US-Americans didn\'t appreciate and celebrate the achievements of the single soldier, but has nothing but distrust for leading officers and politicians. Like every film that defends the war (e.g. \"We Were Soldiers\") also this one avoids the need to give a comprehensible reason for the engagement in South Asia. And for that matter also the reason for every single US-American soldier that was there. Instead, Rambo gets to take revenge for the wounds of a whole nation. It would have been better to work on how to deal with the memories, rather than suppressing them. \"Do we get to win this time?\" Yes, you do.\n\nnegative"

In [21]:
# Return as tensor
tokens = tokenizer(text, return_tensors="pt")
out, state = model.forward(tokens["input_ids"], None)
state[-1].shape # torch.Size([241, 2048]) w/ tensor

torch.Size([241, 2048])

In [22]:
# Return as list
tokens = tokenizer(text)
out, state = model.forward(tokens["input_ids"], None)
state[-1].shape # torch.Size([2048]) w/o tensor

torch.Size([2048])

In [24]:
len(
state)

120

In [25]:
len(state)

120

In [7]:
tokenizer.decode(out.argmax())

' the'

In [8]:
state[-1]

tensor([ 0.2703, -0.3164,  0.1064,  ...,  0.0698, -0.0320, -0.0400],
       device='cuda:0', dtype=torch.float16)

In [9]:
state[-1].shape

torch.Size([2048])

In [10]:
len(state)

120

In [11]:
out, state = model.forward([187, 510, 1563, 310, 247], None)   # use 20B_tokenizer.json
print(out.detach().cpu().numpy())                   # get logits
out, state = model.forward([187, 510], None)
out, state = model.forward([1563], state)           # RNN has state (use deepcopy if you want to clone it)
out, state = model.forward([310, 247], state)

[ -8.828125  -21.828125   -9.359375  ...  -7.7070312  -4.875
  -1.9785156]


In [12]:
from rwkv.utils import PIPELINE, PIPELINE_ARGS
pipeline = PIPELINE(model, "20B_tokenizer.json") 
ctx = "\nIn a shocking finding, scientist discovered a herd of dragons living in a remote, previously unexplored valley, in Tibet. Even more surprising to the researchers was the fact that the dragons spoke perfect Chinese."
print(ctx, end='')

def my_print(s):
    print(s, end='', flush=True)

args = PIPELINE_ARGS(temperature = 1.0, top_p = 0.7, top_k = 100, # top_k = 0 then ignore
                     alpha_frequency = 0.25,
                     alpha_presence = 0.25,
                     token_ban = [0], # ban the generation of some tokens
                     token_stop = [], # stop generation whenever you see any token here
                     chunk_len = 256) # split input into chunks to save VRAM (shorter -> slower)

pipeline.generate(ctx, token_count=200, args=args, callback=my_print)


In a shocking finding, scientist discovered a herd of dragons living in a remote, previously unexplored valley, in Tibet. Even more surprising to the researchers was the fact that the dragons spoke perfect Chinese.

The researchers also found a variety of other reptiles, including tortoises, lizards, snakes and birds.

According to BBC News, scientists have never seen such an incredible collection of species before. They’re calling it the “largest known reptile assemblage ever discovered in the world”.

“It was quite amazing to see all these creatures living in this huge landscape,” said one expert.

Dr Zhenhua Deng from the Chinese Academy of Sciences in Beijing told the BBC that the dragons may be capable of surviving on mountainsides.

“The discovery is very exciting,” he said. “I think it’s an extremely unique find.”

The dragon fossils were discovered by a team of scientists led by biologist Prof Yu Bingbing from Shaanxi Normal University. The team travelled to Tibet to examine a

'\n\nThe researchers also found a variety of other reptiles, including tortoises, lizards, snakes and birds.\n\nAccording to BBC News, scientists have never seen such an incredible collection of species before. They’re calling it the “largest known reptile assemblage ever discovered in the world”.\n\n“It was quite amazing to see all these creatures living in this huge landscape,” said one expert.\n\nDr Zhenhua Deng from the Chinese Academy of Sciences in Beijing told the BBC that the dragons may be capable of surviving on mountainsides.\n\n“The discovery is very exciting,” he said. “I think it’s an extremely unique find.”\n\nThe dragon fossils were discovered by a team of scientists led by biologist Prof Yu Bingbing from Shaanxi Normal University. The team travelled to Tibet to examine a series of caves and outcrops at an altitude of 6,000 metres. The team collected three skeletons, including a'

In [13]:
pipeline.encode(ctx) == GPT2TokenizerFast(tokenizer_file='20B_tokenizer.json').encode(ctx)

True

In [15]:
import os
from rwkv_hf import RWKVModel
from transformers import GPT2TokenizerFast

os.environ["CUDA_VISIBLE_DEVICES"] = "4"

tokenizer = GPT2TokenizerFast(tokenizer_file='20B_tokenizer.json')
text = "I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered controversial I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, even then it's not shot like some cheaply made porno. While my countrymen mind find it shocking, in reality sex and nudity are a major staple in Swedish cinema. Even Ingmar Bergman, arguably their answer to good old boy John Ford, had sex scenes in his films.<br /><br />I do commend the filmmakers for the fact that any sex shown in the film is shown for artistic purposes rather than just to shock people and make money to be shown in pornographic theaters in America. I AM CURIOUS-YELLOW is a good film for anyone wanting to study the meat and potatoes (no pun intended) of Swedish cinema. But really, this film doesn't have much of a plot."
tokens = tokenizer.encode(text)
model = RWKVModel()

RWKV_JIT_ON 1 RWKV_CUDA_ON 0 RESCALE_LAYER 6

Loading /home/kyle/HF-MODEL/rwkv-4-pile-1b5/models--BlinkDL--rwkv-4-pile-1b5/snapshots/6ea995eaa87a17af560c9b41ce1a3d92355c5a49/RWKV-4-Pile-1B5-20220903-8040.pth ...
Strategy: (total 24+1=25 layers)
* cuda [float16, float16], store 25 layers
0-cuda-float16-float16 1-cuda-float16-float16 2-cuda-float16-float16 3-cuda-float16-float16 4-cuda-float16-float16 5-cuda-float16-float16 6-cuda-float16-float16 7-cuda-float16-float16 8-cuda-float16-float16 9-cuda-float16-float16 10-cuda-float16-float16 11-cuda-float16-float16 12-cuda-float16-float16 13-cuda-float16-float16 14-cuda-float16-float16 15-cuda-float16-float16 16-cuda-float16-float16 17-cuda-float16-float16 18-cuda-float16-float16 19-cuda-float16-float16 20-cuda-float16-float16 21-cuda-float16-float16 22-cuda-float16-float16 23-cuda-float16-float16 24-cuda-float16-float16 
emb.weight                        f16      cpu  50277  2048 
blocks.0.ln1.weight               f16   cuda:0   2048       

In [16]:
model(tokens)[-1]

torch.Size([2048])